In [17]:
import pandas as pd
import xlwings as xw
import os
import shutil

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option('display.max_colwidth', None)

df = pd.read_excel('2014.01.21 - Gara Carbonera/ISCRITTI 1 TROFEO CARBONERA 2024.xlsx')
df['CINTURA'] = df['CINTURA'].replace(to_replace='VERDE VE-BL', value='VERDE VE-BLU', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='MARRONE NERA', value='MARRONE e NERA', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='MARRONE E NERA', value='MARRONE e NERA', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='GIALLA GI-ARANCIO', value='GIALLA GI-AR', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='BLU BL-MA', value='BLU BLU-MAR', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='BIANCA e BI-GI', value='BIANCA BI-GI', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='ARANCIO A-V', value='ARANCIO AR-VE', regex=False)
df['CINTURA'] = df['CINTURA'].replace(to_replace='KARATE SACILE FONTANAFREDDA\xa0', value='KARATE SACILE FONTANAFREDDA', regex=False)
df.rename(columns={"SOCIETA'": "SOCIETA"}, inplace=True)
df.drop(columns=['ISCRITTO'], inplace=True)
df.reset_index(drop=True, inplace=True)
df.index.name = 'ID'
df.head()

c:\Users\932197\Documents\dati_locali\git-hub\karate\.venv\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,COGNOME,NOME,ANNO,M / F,CINTURA,SOCIETA,QUOTA DI ISCRIZIONE,FASCIA
ID,,,,,,,,
0,PIERAGNOLO,LEONE,2015,M,VERDE VE-BLU,SCUOLA KARATE MIGNAGOLA,NaN,FASCIA C
1,RONCATO,ELISA,2014,F,GIALLA GI-AR,SCUOLA KARATE MIGNAGOLA,12.0,FASCIA C
2,STEINER,FEDERICO,2015,M,ARANCIO AR-VE,SCUOLA KARATE MIGNAGOLA,12.0,FASCIA C
3,BUCCI,GIOIA,2013,M,GIALLA GI-AR,SCUOLA KARATE MIGNAGOLA,12.0,FASCIA D
4,BUCCI,FRANCESCO,2016,M,GIALLA GI-AR,SCUOLA KARATE MIGNAGOLA,12.0,FASCIA B


In [18]:
df_temp = pd.read_excel("2014.01.21 - Gara Carbonera/Tabellone vuoto.xlsx", sheet_name='Foglio3')
fasce = df_temp['Fascia'].dropna()
anni = df_temp['Anno'].dropna()
categorie = df_temp['Categoria'].dropna()

In [19]:
def ricrea_cartella_tabelloni():
    if os.path.exists('2014.01.21 - Gara Carbonera/Tabelloni generati'):
        shutil.rmtree('2014.01.21 - Gara Carbonera/Tabelloni generati')
    os.makedirs('2014.01.21 - Gara Carbonera/Tabelloni generati')   
    os.makedirs('2014.01.21 - Gara Carbonera/Tabelloni generati/PDF')   
    os.makedirs('2014.01.21 - Gara Carbonera/Tabelloni generati/Excel')   

def iscritti_per_tabellone(fascia, categoria):
    iscritti = df[(df['FASCIA'] == f'FASCIA {fascia}') & (df['CINTURA'] == categoria)].sample(frac=1).sample(frac=1).sample(frac=1).sample(frac=1).sample(frac=1)#.sort_values(by=['SOCIETA'])
    n_iscritti = len(iscritti)
    atleta_singolo = None
    if n_iscritti % 2 == 1:
        # Se sono DISPARI, sceglie l'atleta che non farà il primo incontro fra quelli della società col maggior numero di atleti iscritti
        soc_maggiori_iscritti = iscritti.groupby(['SOCIETA'])[['SOCIETA']].value_counts().idxmax()
        id_atleta_singolo = iscritti[iscritti['SOCIETA'] == soc_maggiori_iscritti].index[0]
        atleta_singolo = iscritti.loc[id_atleta_singolo]
        iscritti.drop(id_atleta_singolo, inplace=True)
    # Ordina gli atleti in modo che non si scontrino due della stessa società, per quanto possibile

    return iscritti, atleta_singolo

def salva_tabellone(app, tab, fascia, categoria):
    tab_vuoto = '2014.01.21 - Gara Carbonera/Tabellone vuoto.xlsx'
    foglio = 'TAB.SINGOLI'
    nome_file = f'FASCIA {fascia} - {categoria}'
    
    wb = app.books.open(tab_vuoto)
    sheet = wb.sheets[foglio]  
    sheet['C1'].value = fascia
    sheet['F1'].value = categoria
    sheet['A5'].value = tab.values
    prima_riga_vuota = 5
    for riga in range(5, 114):
        if (not sheet[f'C{riga}'].value) and (sheet[f'B{riga}'].value == 'R'):
            prima_riga_vuota = riga
            break
    rg = sheet.range(f'{prima_riga_vuota}:114')
    rg.delete(shift='up')
    sheet.to_pdf(f'2014.01.21 - Gara Carbonera/Tabelloni generati/PDF/{nome_file}.pdf')
    wb.save(f'2014.01.21 - Gara Carbonera/Tabelloni generati/Excel/{nome_file}.xlsx')
    wb.close()


In [20]:
ricrea_cartella_tabelloni()
tot_tabelloni = len(fasce) * len(categorie)
n_tab = 1
app = xw.App(visible=False)
for fascia in fasce:
    for categoria in categorie:
        print(f'Tabellone n.{n_tab} di {tot_tabelloni}')
        iscritti, atleta_singolo = iscritti_per_tabellone(fascia, categoria)
        n_incontro = 1
        riga = {}
        tab = pd.DataFrame(columns=['N_INCONTRO', 'CINTURA', 'SOCIETA', 'SOC2', 'SOC3', 'COGNOME', 'NOME'])
        if (not iscritti.empty) or (atleta_singolo is not None):
            for i in range(0, len(iscritti), 2):
                atleta1 = iscritti.iloc[i]
                atleta2 = iscritti.iloc[i + 1]
                tab.loc[len(tab)] = [n_incontro, 'R', atleta1['SOCIETA'].upper(), '', '', atleta1['COGNOME'].upper(), atleta1['NOME'].upper()]
                tab.loc[len(tab)] = [n_incontro, 'B', atleta2['SOCIETA'].upper(), '', '', atleta2['COGNOME'].upper(), atleta2['NOME'].upper()]
                n_incontro += 1
            if atleta_singolo is not None:
                tab.loc[len(tab)] = [n_incontro, 'R', atleta_singolo['SOCIETA'].upper(), '', '', atleta_singolo['COGNOME'].upper(), atleta_singolo['NOME'].upper()]
            salva_tabellone(app, tab, fascia, categoria)
        n_tab += 1
app.quit()

Tabellone n.1 di 30
Tabellone n.2 di 30
Tabellone n.3 di 30
Tabellone n.4 di 30
Tabellone n.5 di 30
Tabellone n.6 di 30
Tabellone n.7 di 30
Tabellone n.8 di 30
Tabellone n.9 di 30
Tabellone n.10 di 30
Tabellone n.11 di 30
Tabellone n.12 di 30
Tabellone n.13 di 30
Tabellone n.14 di 30
Tabellone n.15 di 30
Tabellone n.16 di 30
Tabellone n.17 di 30
Tabellone n.18 di 30
Tabellone n.19 di 30
Tabellone n.20 di 30
Tabellone n.21 di 30
Tabellone n.22 di 30
Tabellone n.23 di 30
Tabellone n.24 di 30
Tabellone n.25 di 30
Tabellone n.26 di 30
Tabellone n.27 di 30
Tabellone n.28 di 30
Tabellone n.29 di 30
Tabellone n.30 di 30
